<a href="https://colab.research.google.com/github/jcorner1/MMH_Project/blob/main/Sat_Image_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

import os
os.listdir("/content/drive/My Drive/Colab Notebooks/subsets/tiles")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['ortho_1-1_hc_s_al127_2023_12.tfw',
 'ortho_1-1_hc_s_al127_2023_11.tfw',
 'ortho_1-1_hc_s_al127_2023_10.tfw',
 'ortho_1-1_hc_s_al127_2023_13.tfw',
 'ortho_1-1_hc_s_al127_2023_12.TIF.ovr',
 'ortho_1-1_hc_s_al127_2023_10.TIF.ovr',
 'ortho_1-1_hc_s_al127_2023_12.TIF',
 'ortho_1-1_hc_s_al127_2023_13.TIF',
 'ortho_1-1_hc_s_al127_2023_11.TIF',
 'ortho_1-1_hc_s_al127_2023_11.TIF.ovr',
 'ortho_1-1_hc_s_al127_2023_13.TIF.ovr',
 'ortho_1-1_hc_s_al127_2023_10.TIF',
 'ortho_1-1_hc_s_al127_2023_13.TIF.aux.xml',
 'ortho_1-1_hc_s_al127_2023_12.TIF.aux.xml',
 'ortho_1-1_hc_s_al127_2023_10.TIF.aux.xml',
 'ortho_1-1_hc_s_al127_2023_11.TIF.aux.xml']

In [3]:
from osgeo import gdal
import numpy as np
gdal.UseExceptions()

def load_sat_image(filename):
  """Loads a satellite image from a file.
  """

  # open the .sid file
  dataset = gdal.Open(filename)

  # get the raster bands
  band1 = dataset.GetRasterBand(1)
  band2 = dataset.GetRasterBand(2)
  band3 = dataset.GetRasterBand(3)

  # read the data as a numpy array
  data1 = band1.ReadAsArray()
  data2 = band2.ReadAsArray()
  data3 = band3.ReadAsArray()

  # create color image with the raster band arrays
  data = np.dstack((data1, data2, data3))
  return data

In [4]:
# prompt: use bicubic compression on the images

import cv2

def compress_bicubic(image):
  """Compresses an image using bicubic interpolation.
  """

  # resize the image using bicubic interpolation
  compressed_image = cv2.resize(image, (image.shape[1] // 2, image.shape[0] // 2), interpolation=cv2.INTER_CUBIC)

  return compressed_image

In [5]:
# iterate through the images
for index_code in [10, 11, 12, 13]:
  img = load_sat_image(f'/content/drive/My Drive/Colab Notebooks/subsets/tiles/ortho_1-1_hc_s_al127_2023_{index_code}.TIF')

  # iterate through x and y
  for x in np.arange(112, img.shape[0], 224):
    for y in np.arange(112, img.shape[1], 224):

      # crop the image
      sub_img = img[x-112:x+112, y-112:y+112]

      # check if image is correct size
      if sub_img.shape == (224, 224, 3):

        if os.path.exists(f'/content/drive/My Drive/Colab Notebooks/Sat_Images/hr_images/ortho_1-1_hc_s_al127_2023_{index_code}_{x}_{y}.npy') == False:
          # compress the sub image using bicubic interpolation
          compressed_image = compress_bicubic(sub_img)

          # save the normal and low resolution image
          np.save(f'/content/drive/My Drive/Colab Notebooks/Sat_Images/hr_images/ortho_1-1_hc_s_al127_2023_{index_code}_{x}_{y}.npy', sub_img)
          np.save(f'/content/drive/My Drive/Colab Notebooks/Sat_Images/lr_images/ortho_1-1_hc_s_al127_2023_{index_code}_{x}_{y}.npy', compressed_image)
